# Load the data

In [6]:
from TF_StateFarm_Loader import *
import numpy as np

NUM_CLASSES = 10 # number of classes that we need
IMAGE_SIZE = 224 # reduce to this size to fit the VGG network
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
LEARNING_RATE = 1e-4
DROPOUT = 0.5
BATCH_SIZE = 200

In [7]:
train_data, train_target = Load_Statefarm_Train()
num_examples = train_data.shape[0] # we have 22424 samples

read train images


In [13]:
rounds = np.floor(num_examples/BATCH_SIZE)
rounds

112.0

In [20]:
import tensorflow as tf

In [ ]:
sess = tf.InteractiveSession()

In [23]:
x = tf.placeholder('float', shape=[BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3])
y_ = tf.placeholder('float', shape=[BATCH_SIZE, NUM_CLASSES])
keep_prob = tf.placeholder('float')

In [ ]:
sess.run(tf.initialize_all_variables())

In [19]:
# Loop through and make the training batches

for i in range(0, int(rounds) ):
    start = i * BATCH_SIZE
    
    if (i < rounds-1) :    
        end = (i + 1) * BATCH_SIZE - 1
    else:
        end = num_examples
    
    Train_Batch_x = train_data[start:end]
    Train_Batch_y = train_target[start:end]
    #print 'Start: %d, End: %d' % (start, end)
    
    # TODO TF on Batch Code
        

### old testing code

In [ ]:
from NN_Components import *

In [ ]:
train_data.shape
train_target.shape

In [ ]:
train_data_s = train_data.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
train_target_s = to_categorical(train_target, NUM_CLASSES)

In [3]:
import NN_Components
#reload(NN_Components)
%autoreload 2

In [ ]:
train_data_s.shape

In [ ]:
Batches = train_data_s.shape[0] / BATCH_SIZE
Batches

In [ ]:
train_target_s.shape

In [13]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [14]:
x = tf.placeholder('float', shape=[BATCH_SIZE, image_width, image_width, 3])
y_ = tf.placeholder('float', shape=[BATCH_SIZE, NUM_CLASSES])
keep_prob = tf.placeholder('float')

# do I initiatlise here?

In [ ]:
#def conv_net(x, weights, biases, dropout):
    
W1 = weight_variable(shape = [3, 3, 3, 64])
b1 = bias_variable(shape = [64])

a1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='SAME')
a2 = tf.nn.relu(a1 + b1)

W2 = weight_variable(shape = [3, 3, 64, 64])
b2 = bias_variable(shape = [64])

a3 = tf.nn.conv2d(a2, W2, strides=[1, 1, 1, 1], padding='SAME')
a4 = tf.nn.relu(a3 + b2)

pooled = max_pool_2x2(a4,colours = 1)

# dense connect
W_fc1 = weight_variable([112 * 112 * 64, 4096])
b_fc1 = bias_variable([4096])

h_pool2_flat = tf.reshape(pooled, [-1, 112*112*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# drop out
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([4096, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())


In [ ]:
for i in range(20000):
  batch = mnist.train.next_batch(50) # need to fix
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [5]:
# model
#SHAPES = [batch, ]
def inference(images):
    """
    Args:
        images: 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
    Returns:
        Logits
    Raises:
        
    """
    #inputshape = [1]
    with tf.variable_scope('conv1') as scope:
        
        # VGG zero pads first
        
        weights = weight_variable([5, 5, 3, 64]) # 5x5 filter across 3 colour channels 64 output
        conv = tf.nn.conv2d(images, weights, strides = [1,2,2,1], padding = 'SAME')
        biases = bias_variable([64])
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name = scope.name)
        # logging here
        
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides = [1,2,2,1], 
                               padding='SAME', name='pool1')
        
        norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                         name = 'norm1')

    return norm1